# Data Transformation of Table "Gênero e Tipo de Declaração"

## Environment Configuration

In [0]:
from pyspark.sql.utils import AnalysisException
from src.utils.udfs import functions_for_df_structure_management as ffdsm

## Data Ingestion from Bronze Layer

In [0]:
df_gender_statement_type = spark.table("brazilian_tax_big_numbers.bronze_layer.delta_genero_e_tipo_de_declaracao")

In [0]:
display(df_gender_statement_type)

## Data Transformation

In [0]:
df_gender_statement_type = df_gender_statement_type.dropna(how='all')

In [0]:
df_casted_gender_statement_type = ffdsm.cast_columns_to_float(df_gender_statement_type, ["AnoCalendario", "Genero", "TipoDeDeclaracao"])

In [0]:
df_casted_gender_statement_type = ffdsm.rename_columns_with_df_name(df_casted_gender_statement_type, "GeneroETipoDeDeclaracao", ["AnoCalendario", "Genero", "TipoDeDeclaracao"])

In [0]:
dbutils.data.summarize(df_casted_gender_statement_type)

In [0]:
df_filled_gender_statement_type = ffdsm.fill_nulls(df_casted_gender_statement_type, ["AnoCalendario", "Genero", "TipoDeDeclaracao"])

## Save as Delta in Silver Layer

In [0]:
spark.sql(f"CREATE SCHEMA IF NOT EXISTS brazilian_tax_big_numbers.silver_layer")

In [0]:
error = None

try:
    df_filled_gender_statement_type.write \
        .mode("overwrite") \
        .saveAsTable(f"brazilian_tax_big_numbers.silver_layer.delta_genero_e_tipo_de_declaracao")
    error = None
except Exception as e:
    error = str(e)
    print(error)